# Extraindo dados de um banco de dados local Postgres, usandos PySpark e SQL e obtendo alguns insigths

By: [Gizélly N.S.](https://www.linkedin.com/in/gizellyns/)   


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DateType,
    TimestampType,
    FloatType,
    DoubleType,
)
import json

from modules.import_conn_properties import import_conn_properties 
from modules.write_on_db import write_on_db
from modules.get_from_db import get_from_db

In [2]:
!pip install psycopg2-binary

In [3]:
spark = (
    SparkSession.builder.appName("01_sql")
    .config("spark.driver.extraClassPath", "postgresql-42.2.10.jar")
    .getOrCreate()
)


## Lendo arquivo em formato CSV e carregando em Spark DataFrame.

In [4]:
def read_clients_from_csv(path: str):
    """ Returns "clientes" dataframe from csv file.

    Args:
        path (str): where is located clientes.csv

    Returns:
        df: spark dataframe  with  "clientes" data
    """    


    client_schema = StructType(
        [
            StructField("id", IntegerType(), True),
            StructField("sexo", StringType(), True),
            StructField("data_nascimento", DateType(), True),
            StructField("data_cadastro", TimestampType(), True),
            StructField("cidade", StringType(), True),
            StructField("sigla", StringType(), True),
        ]
    )

    df = (
        spark.read.format("csv")
        .option("header", "true")
        .option("encoding", "UTF-8")
        .load(path, schema=client_schema)
    )

    return df

df_clients = read_clients_from_csv('./files/clientes.csv')

In [5]:
df_clients.show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 40554|null|     1984-03-12|2020-07-31 03:30:26|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 7413

In [6]:
def read_results_from_csv(path:str):
    """ Returns "resultado" dataframe from csv file.

    Args:
        path (str): where is located resultado.csv

    Returns:
        df: spark dataframe  with  "resultado" data
    """    
    client_schema = StructType(
        [
            StructField("data_acesso", TimestampType(), True),
            StructField("clientes_id", IntegerType(), True),
            StructField("buyin", FloatType(), True),
            StructField("rake", DoubleType(), True),
            StructField("winning", DoubleType(), True)
        ]
    )

    df = (
        spark.read.format("csv")
        .option("header", "true")
        .option("encoding", "UTF-8")
        .load(path, schema=client_schema)
    )

    return df


df_results = read_results_from_csv('./files/resultado.csv')

In [7]:
df_results.show()

+-------------------+-----------+-----+-------------------+-------------------+
|        data_acesso|clientes_id|buyin|               rake|            winning|
+-------------------+-----------+-----+-------------------+-------------------+
|2022-08-14 00:00:00|        505| 25.0|               1.75|              -2.75|
|2022-08-15 00:00:00|        505| 20.0|                4.0|  8.399999618530273|
|2022-08-16 00:00:00|        505| 65.0|  5.400000035762787|-19.300000309944153|
|2022-08-17 00:00:00|        505| 30.0| 2.6500000953674316|              -30.0|
|2022-08-19 00:00:00|        505|  5.0| 0.3499999940395355|  3.549999952316284|
|2022-08-20 00:00:00|        505| 15.0|  1.899999976158142|               0.25|
|2022-08-23 00:00:00|        505| 10.0|  2.950000047683716|-3.0999999046325684|
|2022-08-30 00:00:00|        505|  5.0|0.30000001192092896|               -5.0|
|2022-11-20 00:00:00|        505| 43.0|    7.3999999538064|              -33.0|
|2022-11-21 00:00:00|        505| 15.0| 

## Criando Tabelas no Postgress

In [8]:
%run -i './sql/create_table_clientes.py'

Tabela clients criada com sucesso.


In [9]:
%run -i './sql/create_table_results.py'

Tabela results criada com sucesso.


## Escrevendo no banco de dados: Clientes

In [10]:
clients_properties = import_conn_properties('./conn_properties/clients_properties.txt')

In [11]:
write_on_db(spark, df_clients, clients_properties) 

'Sucess'

### Escrevendo no banco de dados: Resultados

In [12]:
results_properties = import_conn_properties('./conn_properties/results_properties.txt')

In [13]:
write_on_db(spark, df_results, results_properties)

'Sucess'

## Lendo os dados inseridos no banco de dados para fazer as análises, guardando o retorno em tabelas temporárias em tempo de execução.

In [14]:
get_from_db(spark, clients_properties, "clientes")

spark.sql("SELECT  * FROM clientes").show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 40554|null|     1984-03-12|2020-07-31 03:30:26|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 7413

In [15]:
get_from_db(spark, results_properties, "resultado")

spark.sql("SELECT  * FROM clientes").show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 40554|null|     1984-03-12|2020-07-31 03:30:26|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 7413

## Respondendo a perguntas

### Quanto de rake foi gerado por cada Geração de jogadores?

Cada geração tendo o seguinte critério:

* Veteranos, geração formada por pessoas que nasceram entre 1925 e 1940.
* Baby Boomers são os nascidos entre 1941 e 1959.
* Geração X, que compreende o período de 1960 a 1979.
* Geração Y é composta por indivíduos que nasceram entre 1980 e 1995.
* Geração Z é composta com os nascidos a partir de 1996 até 2010.
* Geração Alpha, engloba todos os nascidos a partir de 2010 até a presente data.

In [16]:
rake_by_generation = spark.sql("SELECT  \
                                   SUM(r.rake) FILTER (WHERE c.data_nascimento BETWEEN '1941-01-01' AND '1959-12-31') AS baby_boomers, \
                                   SUM(r.rake) FILTER (WHERE c.data_nascimento BETWEEN '1960-01-01' AND '1979-12-31') AS geracao_x, \
                                   SUM(r.rake) FILTER (WHERE c.data_nascimento BETWEEN '1980-01-01' AND '1995-12-31') AS geracao_y, \
                                   SUM(r.rake) FILTER (WHERE c.data_nascimento BETWEEN '1996-01-01' AND '2009-12-31') AS geracao_z, \
                                   SUM(r.rake) FILTER (WHERE c.data_nascimento >= '2010-01-01') as geracao_alpha \
                               FROM \
                                    clientes c \
                                INNER JOIN resultado r ON c.id = r.clientes_id GROUP BY r.rake\
                                ")


rake_by_generation.select(F.sum(rake_by_generation.baby_boomers).alias("baby_boomers_total_rake"),
                          F.sum(rake_by_generation.geracao_x).alias("geracao_x_total_rake"),
                          F.sum(rake_by_generation.geracao_y).alias("geracao_y_total_rake"),
                          F.sum(rake_by_generation.geracao_z).alias("geracao_z_total_rake"),
                          F.sum(rake_by_generation.geracao_alpha).alias("geracao_alpha_total_rake")).show()



+-----------------------+--------------------+--------------------+--------------------+------------------------+
|baby_boomers_total_rake|geracao_x_total_rake|geracao_y_total_rake|geracao_z_total_rake|geracao_alpha_total_rake|
+-----------------------+--------------------+--------------------+--------------------+------------------------+
|   38549.66000577434...|520650.7700801343...|612412.5099798794...|93378.69005352071...|                    null|
+-----------------------+--------------------+--------------------+--------------------+------------------------+



* Observação: Não há dados sobre veteranos, geração formada por pessoas que nasceram entre 1925 e 1940 e o total rake para  Geração Alpha é inexistente na base de dados.

### Qual foi o rake gerado por mês?

In [17]:
dt_acesso = spark.sql("SELECT data_acesso \
FROM resultado \
GROUP BY data_acesso \
ORDER BY data_acesso DESC \
")
dt_acesso.createOrReplaceTempView("data_acesso")

rake_por_mes = spark.sql("SELECT EXTRACT(MONTH FROM data_acesso) AS mes, \
EXTRACT(YEAR FROM data_acesso) AS year, \
SUM(rake) AS total_rake \
FROM resultado \
GROUP BY EXTRACT(MONTH FROM data_acesso), EXTRACT(YEAR FROM data_acesso) \
ORDER BY year, mes \
").show()

+---+----+--------------------+
|mes|year|          total_rake|
+---+----+--------------------+
|  6|2020|9349.120016444473...|
|  7|2020|12373.77001768350...|
|  8|2020|18130.99000354483...|
|  9|2020|11097.09996528737...|
| 10|2020|7260.410003677009...|
| 11|2020|7699.459994543344...|
| 12|2020|7708.860011853279...|
|  1|2021|10875.24000201374...|
|  2|2021|15381.23000405542...|
|  3|2021|28560.31008905358...|
|  4|2021|63925.37990119870...|
|  5|2021|10196.66995991766...|
|  6|2021|7312.099979553371...|
|  7|2021|8615.850006807594...|
|  8|2021|7137.780008828268...|
|  9|2021|15338.07001414522...|
| 10|2021|24094.74004009738...|
| 11|2021|26819.62997406721...|
| 12|2021|36064.35995101556...|
|  1|2022|40423.61998029797...|
+---+----+--------------------+
only showing top 20 rows



### Qual sexo tem uma maior proporção de ganhadores?
Como ganhador, considere um jogador com Winning maior que 0

In [18]:
winning_genero = spark.sql("SELECT SUM(r.winning) FILTER (WHERE r.winning > 0)AS total_winning, \
                            SUM(r.winning) FILTER (WHERE c.sexo = 'm' AND r.winning > 0) AS total_masculino, \
                            SUM(r.winning)  FILTER (WHERE c.sexo = 'f' AND r.winning > 0) AS total_feminino \
                            FROM \
                                clientes c \
                            INNER JOIN resultado r ON c.id = r.clientes_id GROUP BY r.winning\
                            ")

winning_genero.select((F.sum(winning_genero.total_feminino)/F.sum(winning_genero.total_winning)*100).alias("proporcao_winning_feminino (%)"),
                     (F.sum(winning_genero.total_masculino)/F.sum(winning_genero.total_winning)*100).alias("proporcao_winning_masculino (%)")).show()



+------------------------------+-------------------------------+
|proporcao_winning_feminino (%)|proporcao_winning_masculino (%)|
+------------------------------+-------------------------------+
|                      7.098200|                      84.433700|
+------------------------------+-------------------------------+

